In [ ]:
!rm -rf sample_data/

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from transformers import LlavaForConditionalGeneration, LlavaProcessor, AutoProcessor
from transformers import BitsAndBytesConfig
import torch
import json
import random
from PIL import Image, ImageDraw
import os
import argparse
import re

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_compute_dtype=torch.float16
# )

processor = AutoProcessor.from_pretrained('llava-hf/llava-1.5-7b-hf')
model = LlavaForConditionalGeneration.from_pretrained(
    'llava-hf/llava-1.5-7b-hf',
    quantization_config=quantization_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

# Fine-tuning - HuggingFace PEFT

In [ ]:
!unzip /content/val_images.zip

Archive:  /content/val_images.zip
   creating: val_images/
  inflating: __MACOSX/._val_images   
   creating: val_images/2207.06989v1/
  inflating: __MACOSX/val_images/._2207.06989v1  
   creating: val_images/2006.14567v3/
  inflating: __MACOSX/val_images/._2006.14567v3  
   creating: val_images/2106.13531v1/
  inflating: __MACOSX/val_images/._2106.13531v1  
   creating: val_images/2008.13719v2/
  inflating: __MACOSX/val_images/._2008.13719v2  
   creating: val_images/1903.05854v1/
  inflating: __MACOSX/val_images/._1903.05854v1  
   creating: val_images/1908.06605v2/
  inflating: __MACOSX/val_images/._1908.06605v2  
   creating: val_images/2010.08865v1/
  inflating: __MACOSX/val_images/._2010.08865v1  
   creating: val_images/1907.05012v2/
  inflating: __MACOSX/val_images/._1907.05012v2  
   creating: val_images/2206.04153v2/
  inflating: __MACOSX/val_images/._2206.04153v2  
   creating: val_images/1904.10637v1/
  inflating: __MACOSX/val_images/._1904.10637v1  
   creating: val_images

In [ ]:
import pandas as pd

df = pd.read_csv('/content/sample_val.csv')
df

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer
0,1705.08868v2,Which objective function resulted in the highe...,ADV,1705.08868v2-Table1-1.png,Best MODE scores and test negative log-likeli...,NaN
1,1705.08868v2,Which objective function achieved the best Inc...,The ADV objective function achieved the best I...,1705.08868v2-Table2-1.png,Best Inception scores and test negative logli...,NaN
2,1705.08868v2,Do Gaussian Mixture Models or adversarially le...,Gaussian Mixture Models.,1705.08868v2-Figure3-1.png,Gaussian Mixture Models outperform adversaria...,NaN
3,1705.08868v2,Which method produced the generator with the m...,The hybrid method.,1705.08868v2-Figure4-1.png,CDF of the singular values magnitudes for the...,NaN
4,1705.08868v2,Which objective function resulted in the lowes...,The hybrid objective function resulted in the ...,1705.08868v2-Table3-1.png,Comparison of inference techniques for negati...,NaN
...,...,...,...,...,...,...
2080,2310.17594v2,Which method performs the best on average acro...,SPA (Ours),2310.17594v2-Table6-1.png,Classification Accuracy (%) on DomainNet126 f...,NaN
2081,2310.17594v2,Which method appears to have the best domain a...,SPA,2310.17594v2-Figure2-1.png,Feature Visualization. the t-SNE plot of DANN...,NaN
2082,2310.17594v2,Which method seems to have the best domain ali...,SPA,2310.17594v2-Figure5-1.png,Feature Visualization. the t-SNE plot of DANN...,NaN
2083,2310.17594v2,"Which of the four methods, DANN, BSP, NPL, or ...",SPA,2310.17594v2-Figure6-1.png,Feature Visualization. the t-SNE plot of DANN...,NaN


In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType

# Enable gradient checkpointing and prepare model for LoRA
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Inject LoRA adapters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 42,336,256 || all params: 7,105,763,328 || trainable%: 0.5958


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

class LLaVADataset(Dataset):
    def __init__(self, df, processor):
        """
        Args:
          df: A pandas DataFrame with columns: 'paper', 'reference_figure', 'question', and 'answer'
          processor: The LlavaProcessor instance loaded from pretrained
          max_length: The maximum sequence length (should be large enough to accommodate prompt and answer)
        """
        self.df = df
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get one row from the dataset.
        row = self.df.iloc[idx]
        image_path = f"/content/val_images/{row['paper']}/{row['reference_figure']}"
        image = Image.open(image_path).convert("RGB")

        # Build the prompt: include the image placeholder and question.
        prompt = f"<image> The image below contains visual details that are crucial to answer the following question. Please analyze the visual content present in the image thoroughly to derive the best possible answer.\n### Question: {row['question']}\n### Answer:"

        # Create a unified sequence by appending the answer after the prompt.
        full_text = prompt + " " + row["answer"]

        # Tokenize the full sequence using the processor.
        inputs = self.processor(
            images=image,
            text=full_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=1024,
            do_rescale=False  # Adjust if your images are in a specific range
        )

        # To compute loss only on the answer tokens, we mask out the prompt portion.
        # 1. Tokenize the prompt alone to know its length.
        prompt_ids = self.processor.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024
        ).input_ids.squeeze(0)

        # 2. Copy the tokenized full sequence to create labels.
        full_ids = inputs["input_ids"]
        labels = full_ids.clone()

        # 3. Mask the prompt tokens by setting them to -100.
        prompt_length = prompt_ids.shape[0]
        labels[:, :prompt_length] = -100

        # 4. Insert the labels back into inputs.
        inputs["labels"] = labels[0]  # remove extra batch dimension

        # Remove extra dimensions (squeeze the batch dimension)
        return {k: v.squeeze(0) for k, v in inputs.items()}


df = pd.read_csv('/content/sample_val.csv')
dataset = LLaVADataset(df, processor)

for sample in dataset:
    for key, val in sample.items():
        if hasattr(val, "shape"):
            print(f"{key}: {val.shape}")
        else:
            print(f"{key}: {val}")
    break

input_ids: torch.Size([1024])
attention_mask: torch.Size([1024])
pixel_values: torch.Size([3, 336, 336])
labels: torch.Size([1024])


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llava-lora-ft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

model.config.label_names = ["labels"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
500,14.926500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=520, training_loss=14.886317033034105, metrics={'train_runtime': 14912.1306, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.035, 'total_flos': 1.783303075088302e+17, 'train_loss': 14.886317033034105, 'epoch': 1.99712368168744})

In [ ]:
model.save_pretrained("llava-lora-adapter")
processor.save_pretrained("llava-lora-adapter")

['llava-lora-adapter/processor_config.json']

In [ ]:
!zip -r /content/llava-lora-adapter.zip /content/llava-lora-adapter

  adding: content/llava-lora-adapter/ (stored 0%)
  adding: content/llava-lora-adapter/preprocessor_config.json (deflated 49%)
  adding: content/llava-lora-adapter/tokenizer_config.json (deflated 74%)
  adding: content/llava-lora-adapter/adapter_model.safetensors (deflated 10%)
  adding: content/llava-lora-adapter/special_tokens_map.json (deflated 78%)
  adding: content/llava-lora-adapter/tokenizer.json (deflated 85%)
  adding: content/llava-lora-adapter/tokenizer.model (deflated 55%)
  adding: content/llava-lora-adapter/chat_template.json (deflated 61%)
  adding: content/llava-lora-adapter/processor_config.json (deflated 27%)
  adding: content/llava-lora-adapter/added_tokens.json (deflated 20%)
  adding: content/llava-lora-adapter/README.md (deflated 66%)
  adding: content/llava-lora-adapter/adapter_config.json (deflated 56%)


In [ ]:
from google.colab import files
files.download('/content/llava-lora-adapter.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load Model

In [ ]:
!unzip /content/llava-lora-adapter.zip
!ls /content/content/llava-lora-adapter/

Archive:  /content/llava-lora-adapter.zip
   creating: content/llava-lora-adapter/
  inflating: content/llava-lora-adapter/preprocessor_config.json  
  inflating: content/llava-lora-adapter/tokenizer_config.json  
  inflating: content/llava-lora-adapter/adapter_model.safetensors  
  inflating: content/llava-lora-adapter/special_tokens_map.json  
  inflating: content/llava-lora-adapter/tokenizer.json  
  inflating: content/llava-lora-adapter/tokenizer.model  
  inflating: content/llava-lora-adapter/chat_template.json  
  inflating: content/llava-lora-adapter/processor_config.json  
  inflating: content/llava-lora-adapter/added_tokens.json  
  inflating: content/llava-lora-adapter/README.md  
  inflating: content/llava-lora-adapter/adapter_config.json  
adapter_config.json	   preprocessor_config.json  tokenizer_config.json
adapter_model.safetensors  processor_config.json     tokenizer.json
added_tokens.json	   README.md		     tokenizer.model
chat_template.json	   special_tokens_map.json


In [ ]:
from peft import PeftModel

save_path = "/content/content/llava-lora-adapter/"

processor = AutoProcessor.from_pretrained(save_path)
model = PeftModel.from_pretrained(model, save_path)

# Inference

In [ ]:
!unzip /content/SPIQA_testA_Images.zip

Archive:  /content/SPIQA_testA_Images.zip
   creating: SPIQA_testA_Images/
   creating: SPIQA_testA_Images/1809.03449v3/
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure4-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure2-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table2-1.png  
   creating: SPIQA_testA_Images/1710.01507v4/
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Table1-1.png  
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Figure1-1.png  
   creating: SPIQA_testA_Images/1709.02755v5/
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02755v5-Table1-1.png  
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02

In [ ]:
import pandas as pd

df = pd.read_csv('/content/sample.csv')
df

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,NaN
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,NaN
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,NaN
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,NaN
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,NaN
...,...,...,...,...,...,...
661,1809.04276v2,How does the discriminator in the proposed REA...,The discriminator takes as input a response an...,1809.04276v2-Figure1-1.png,Figure 1: An overview of our proposed approach...,NaN
662,1710.01507v4,What is the role of the LSTM network in the mo...,The LSTM network is used to process the post t...,1710.01507v4-Figure1-1.png,Model Architecture,NaN
663,1709.00139v4,Which method generally achieved a lower object...,"For all datasets presented, Incremental SVM ac...",1709.00139v4-Table1-1.png,Table 1: Experimental Results of FISVDD and In...,NaN
664,1809.01989v2,Which method achieved the highest tracking acc...,The Ridge method achieved the lowest sum of ab...,1809.01989v2-Table1-1.png,Table 1. Absolute percentage errors for differ...,NaN


In [ ]:
def generate_response(image_path, question, return_full=False):
    image = Image.open(image_path).convert("RGB")
    # display(image)
    prompt = f"""The image contains relevant information needed to answer the below question.
    Answer this question: {question}"""
    # prompt = f"<image> The image below contains visual details that are crucial to answer the following question. Please analyze the visual content present in the image thoroughly to derive the best possible answer.\n### Question: {question}\n### Answer:"

    """Generate response from model"""
    conversation = [{
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ],
    }]

    inputs = processor.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to(model.device, torch.float16)

    generate_ids = model.generate(**inputs, max_new_tokens=200)
    full_response = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]
    return full_response if return_full else full_response.split("ASSISTANT:")[-1].strip()

idx = 1
image_path = f"/content/SPIQA_testA_Images/{df.iloc[idx]['paper']}/{df.iloc[idx]['reference_figure']}"
question = df.iloc[idx]['question']
answer = generate_response(image_path, question)
print("Actual Answer:", df.iloc[idx]['answer'])
print("Generated Answer:", answer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Actual Answer: The KEHNN model performs the best for response selection. This is evident because it achieves the highest scores across all metrics (R$2$@1, R${10}$@1, R${10}$@2, and R${10}$@5) compared to all other models in the table.
Generated Answer: The model that performs the best for response selection is the "Ours" model. We can tell this because the "Ours" model has the highest accuracy for both the "all" and "top" response selection tasks. This is shown in the table, where the "Ours" model has the highest accuracy for both tasks. Additionally, the "Ours" model has the highest F1 score for both tasks, which further supports that it performs the best for response selection.


In [ ]:
from tqdm import tqdm

answers = []
for idx, val in tqdm(df.iterrows(), total=len(df)):
    image_path = f"/content/SPIQA_testA_Images/{df.iloc[idx]['paper']}/{df.iloc[idx]['reference_figure']}"
    question = df.iloc[idx]['question']
    answer = generate_response(image_path, question)
    # print("Question:", question)
    # print("Actual Answer:", df.iloc[idx]['answer'])
    # print("Generated Answer:", answer)
    answers.append(answer)

100%|██████████| 666/666 [2:12:40<00:00, 11.95s/it]


In [ ]:
df['generated_answer'] = answers
# df.to_csv(f"llava_quantized_zero_shot_results.csv", index=False)
df.to_csv(f"llava_quantized_fine_tuned_results.csv", index=False)

In [ ]:
from google.colab import files
files.download('/content/llava_quantized_fine_tuned_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>